In [1]:
import os
from groq import Groq
from dotenv import load_dotenv
from typing import List, Optional
from prompts import base, content, condensed
from pydantic import BaseModel, Field, RootModel

load_dotenv()

True

In [2]:
from pydantic import BaseModel
from typing import List, Optional


class Event(BaseModel):
    event_id: str
    date: str
    title: str
    start_time: str
    end_time: Optional[str] = None
    duration: Optional[int] = None
    host: Optional[List[str]] | str = None
    location: str
    categories: List[str]
    description: Optional[str] = None


class EventItinerary(BaseModel):
    events: List[Event]

In [3]:
client = Groq(
    api_key=os.getenv("GROQ_API_KEY"),
)

In [4]:
resp = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {
            "role": "system",
            "content": condensed,
        },
        {
            "role": "user",
            "content": content,
        },
    ],
    # response_format={"type": "json_object"},
    temperature=0,
    max_tokens=8000,
)

In [5]:
print(resp)

ChatCompletion(id='chatcmpl-b63b6c60-f1a0-442f-96e4-cbd7c7162c97', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="**Event Analysis:**\n\n**E001**\nDate: 27th April 2022\nTitle: March Past\nTime: 5:00-5:30 PM\nDuration: 30 minutes\nHost: N/A\nLocation: Football Ground\nCategories: Sports, March Past, Opening Ceremony\nDescription: The opening event of Armotsav 2K22, a march past ceremony.\n\n**E002**\nDate: 27th April 2022\nTitle: 100 m Heats Boys and Girls\nTime: 5:30-6:00 PM\nDuration: 30 minutes\nHost: N/A\nLocation: Football Ground\nCategories: Sports, Athletics, Track and Field\nDescription: A 100m heats event for boys and girls.\n\n**E003**\nDate: 27th April 2022\nTitle: Chess Boys and Girls\nTime: 5:30-6:30 PM\nDuration: 60 minutes\nHost: N/A\nLocation: Amphitheatre\nCategories: Sports, Chess, Board Games\nDescription: A chess tournament for boys and girls.\n\n**E004**\nDate: 27th April 2022\nTitle: Football (Match -1)\nTime: 

In [6]:
print(resp.choices[0].message.content)

**Event Analysis:**

**E001**
Date: 27th April 2022
Title: March Past
Time: 5:00-5:30 PM
Duration: 30 minutes
Host: N/A
Location: Football Ground
Categories: Sports, March Past, Opening Ceremony
Description: The opening event of Armotsav 2K22, a march past ceremony.

**E002**
Date: 27th April 2022
Title: 100 m Heats Boys and Girls
Time: 5:30-6:00 PM
Duration: 30 minutes
Host: N/A
Location: Football Ground
Categories: Sports, Athletics, Track and Field
Description: A 100m heats event for boys and girls.

**E003**
Date: 27th April 2022
Title: Chess Boys and Girls
Time: 5:30-6:30 PM
Duration: 60 minutes
Host: N/A
Location: Amphitheatre
Categories: Sports, Chess, Board Games
Description: A chess tournament for boys and girls.

**E004**
Date: 27th April 2022
Title: Football (Match -1)
Time: 6:15-7:00 PM
Duration: 45 minutes
Host: N/A
Location: Football Ground
Categories: Sports, Football, Team Sports
Description: The first football match of Armotsav 2K22.

**E005**
Date: 28th April 2022
Tit

In [7]:
thing_content = resp.choices[0].message.content

In [8]:
json_thing = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {
            "role": "system",
            "content": base,
        },
        {
            "role": "user",
            "content": thing_content,
        },
    ],
    response_format={"type": "json_object"},
    temperature=0,
    max_tokens=8000,
)

In [9]:
print(json_thing.choices[0].message.content)

{
   "events":[
      {
         "event_id":"E001",
         "date":"27th April 2022",
         "title":"March Past",
         "start_time":"5:00 PM",
         "end_time":"5:30 PM",
         "duration":30,
         "host":null,
         "location":"Football Ground",
         "categories":[
            "Sports",
            "March Past",
            "Opening Ceremony"
         ],
         "description":"The opening event of Armotsav 2K22, a march past ceremony."
      },
      {
         "event_id":"E002",
         "date":"27th April 2022",
         "title":"100 m Heats Boys and Girls",
         "start_time":"5:30 PM",
         "end_time":"6:00 PM",
         "duration":30,
         "host":null,
         "location":"Football Ground",
         "categories":[
            "Sports",
            "Athletics",
            "Track and Field"
         ],
         "description":"A 100m heats event for boys and girls."
      },
      {
         "event_id":"E003",
         "date":"27th April 2022",
 

In [10]:
import json


json_thing_content = json.loads(json_thing.choices[0].message.content)

In [11]:
from pydantic import ValidationError


def validate_json(json_data):
    try:
        event_itinerary = EventItinerary.model_validate(json_data)
        print("Validation successful!")
        return event_itinerary
    except ValidationError as e:
        print("Validation failed!")
        print(e.json())
        return None

In [12]:
validate_json(json_thing_content)

Validation successful!


EventItinerary(events=[Event(event_id='E001', date='27th April 2022', title='March Past', start_time='5:00 PM', end_time='5:30 PM', duration=30, host=None, location='Football Ground', categories=['Sports', 'March Past', 'Opening Ceremony'], description='The opening event of Armotsav 2K22, a march past ceremony.'), Event(event_id='E002', date='27th April 2022', title='100 m Heats Boys and Girls', start_time='5:30 PM', end_time='6:00 PM', duration=30, host=None, location='Football Ground', categories=['Sports', 'Athletics', 'Track and Field'], description='A 100m heats event for boys and girls.'), Event(event_id='E003', date='27th April 2022', title='Chess Boys and Girls', start_time='5:30 PM', end_time='6:30 PM', duration=60, host=None, location='Amphitheatre', categories=['Sports', 'Chess', 'Board Games'], description='A chess tournament for boys and girls.'), Event(event_id='E004', date='27th April 2022', title='Football (Match -1)', start_time='6:15 PM', end_time='7:00 PM', duration=